# TOR html content ai or human detection

In [ ]:
# used pyenv 3/10/15 env 
!pip install transformers
!pip install tf-keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 1.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 369.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.1/447.1 kB 298.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 791.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 970.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 2.4 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

In [3]:
!pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 657.6 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.3 MB/s eta 0:00:00
  Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

# ۱. تنظیمات مسیر و پروکسی
FILE_PATH = "/Users/kiananasiri/Downloads/TorBot/collected_data_v10/page3.html"
OUTPUT_DIR = "html_dataset_100"
TARGET_COUNT = 100
proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# ۲. خواندن فایل و استخراج لینک‌ها
def get_urls_from_file(path):
    print(f"Reading links from: {path}")
    urls = []
    try:
        with open(path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            # استخراج تمام متن‌هایی که با http شروع شده و شامل .onion هستند
            # (چون فایل شما لینک‌ها را به صورت متن ساده یا داخل تگ <a> دارد)
            all_text = soup.get_text(separator=' ')
            words = all_text.split()
            for word in words:
                if ".onion" in word and word.startswith("http"):
                    # تمیز کردن کاراکترهای اضافه مثل <br> یا فضای خالی
                    clean_url = word.strip().replace('<br>', '').replace(',', '')
                    if clean_url not in urls:
                        urls.append(clean_url)
    except Exception as e:
        print(f"Error reading file: {e}")
    return urls

# ۳. تابع دانلود تکی
def download_site(url, index):
    try:
        print(f"[{index}] Fetching: {url}")
        r = requests.get(url, proxies=proxies, timeout=25, verify=False)
        if r.status_code == 200 and len(r.text) > 100:
            filename = f"site_{index}.html"
            with open(os.path.join(OUTPUT_DIR, filename), "w", encoding='utf-8') as f:
                f.write(r.text)
            return True
    except:
        pass
    return False

# ۴. اجرای اصلی
if __name__ == "__main__":
    all_urls = get_urls_from_file(FILE_PATH)
    print(f"Found {len(all_urls)} onion links in your file.")
    
    # محدود کردن به لینک‌های موجود یا ۱۰۰ عدد
    urls_to_crawl = all_urls[:200] # ۲۰۰ تا برمی‌داریم که حتماً ۱۰۰ تا موفق داشته باشیم
    
    success_count = 0
    # استفاده از ThreadPool برای سرعت (۱۰ دانلود همزمان)
    print("Starting Multi-threaded download...")
    with ThreadPoolExecutor(max_workers=10) as executor:
        # ارسال لینک‌ها به تردها
        results = list(executor.map(lambda p: download_site(p[1], p[0]), enumerate(urls_to_crawl)))
        success_count = sum(1 for x in results if x)

    print(f"\n--- DONE ---")
    print(f"Total links processed: {len(urls_to_crawl)}")
    print(f"Successfully saved: {success_count} files in '{OUTPUT_DIR}'")


In [4]:
import os
from bs4 import BeautifulSoup
from transformers import pipeline

/Users/kiananasiri/.pyenv/versions/3.10.15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import os
from bs4 import BeautifulSoup
from transformers import pipeline

print("Loading Neural Network (RoBERTa)...")
detector = pipeline("text-classification", model="roberta-base-openai-detector")


Loading Neural Network (RoBERTa)...


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [ ]:
import os
from bs4 import BeautifulSoup
from transformers import pipeline

def analyze_dataset(folder_path):
    results = {"AI": 0, "Human": 0}
    
    # اسکن تمام فایل‌های HTML شما
    files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    print(f"Analyzing {len(files)} files...")

    for filename in files:
        path = os.path.join(folder_path, filename)
        with open(path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            # استخراج متن خالص
            text = soup.get_text()
            # مدل‌های تشخیص معمولاً روی ۵۱۲ توکن اول بهترین جواب را می‌دهند
            clean_text = " ".join(text.split())

            if len(clean_text) > 100: # فایل‌های خیلی کوچک را نادیده بگیر
                prediction = detector(clean_text)[0]
                label = "Human" if prediction['label'] == 'Real' else "AI"
                results[label] += 1
                print(f"File: {filename} | Prediction: {label} ({prediction['score']:.2%})")

    print("\n--- FINAL STATISTICS ---")
    print(f"Detected as AI: {results['AI']}")
    print(f"Detected as Human: {results['Human']}")

# مسیر پوشه‌ای که ۱۰۰ فایل در آن است را اینجا بدهید
analyze_dataset("./html_dataset_100")

/Users/kiananasiri/.pyenv/versions/3.10.15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Neural Network (RoBERTa)...


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


Analyzing 168 files...
File: site_92.html | Prediction: Human (99.98%)
File: site_147.html | Prediction: Human (98.56%)
File: site_110.html | Prediction: Human (98.67%)
File: site_84.html | Prediction: Human (99.98%)
File: site_151.html | Prediction: Human (99.98%)
File: site_10.html | Prediction: Human (99.97%)
File: site_47.html | Prediction: Human (99.93%)
File: site_51.html | Prediction: AI (53.64%)
File: site_184.html | Prediction: Human (99.30%)
File: site_71.html | Prediction: Human (99.98%)
File: site_26.html | Prediction: Human (96.75%)
File: site_67.html | Prediction: Human (98.96%)
File: site_126.html | Prediction: Human (95.90%)
File: site_171.html | Prediction: Human (99.98%)
File: site_167.html | Prediction: Human (99.73%)
File: site_188.html | Prediction: AI (88.38%)
File: site_130.html | Prediction: Human (99.91%)
File: site_131.html | Prediction: Human (98.75%)
File: site_166.html | Prediction: Human (99.88%)
File: site_170.html | Prediction: Human (99.98%)
File: site_

In [ ]:

def analyze_dataset(folder_path):
    results = {"AI": 0, "Human": 0}
    
    # اسکن تمام فایل‌های HTML شما
    files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    print(f"Analyzing {len(files)} files...")

    for filename in files:
        path = os.path.join(folder_path, filename)
        with open(path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            # استخراج متن خالص
            text = soup.get_text()
            # مدل‌های تشخیص معمولاً روی ۵۱۲ توکن اول بهترین جواب را می‌دهند
            clean_text = " ".join(text.split())[:1000]

            if len(clean_text) > 100: # فایل‌های خیلی کوچک را نادیده بگیر
                prediction = detector(clean_text)[0]
                label = "Human" if prediction['label'] == 'Real' else "AI"
                results[label] += 1
                print(f"File: {filename} | Prediction: {label} ({prediction['score']:.2%})")

    print("\n--- FINAL STATISTICS ---")
    print(f"Detected as AI: {results['AI']}")
    print(f"Detected as Human: {results['Human']}")

# مسیر پوشه‌ای که ۱۰۰ فایل در آن است را اینجا بدهید
analyze_dataset("./html_dataset_100")

Analyzing 168 files...
File: site_92.html | Prediction: Human (99.98%)
File: site_147.html | Prediction: Human (98.56%)
File: site_110.html | Prediction: Human (98.67%)
File: site_84.html | Prediction: Human (99.98%)
File: site_151.html | Prediction: Human (99.98%)
File: site_10.html | Prediction: Human (99.97%)
File: site_47.html | Prediction: Human (99.93%)
File: site_51.html | Prediction: AI (53.64%)
File: site_184.html | Prediction: Human (99.30%)
File: site_71.html | Prediction: Human (99.98%)
File: site_26.html | Prediction: Human (96.75%)
File: site_67.html | Prediction: Human (98.96%)
File: site_126.html | Prediction: Human (95.90%)
File: site_171.html | Prediction: Human (99.98%)
File: site_167.html | Prediction: Human (99.73%)
File: site_188.html | Prediction: AI (88.38%)
File: site_130.html | Prediction: Human (99.91%)
File: site_131.html | Prediction: Human (98.75%)
File: site_166.html | Prediction: Human (99.88%)
File: site_170.html | Prediction: Human (99.98%)
File: site_

In [5]:
!pip install pyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 1.7 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import fitz  # PyMuPDF
from transformers import pipeline

# استفاده از یک مدل چندزبانه که برای تشخیص متن (Fake/Real) منعطف‌تر است
# نکته: مدل‌های عمومی فقط ویژگی‌های زبانی را استخراج می‌کنند. 
# برای تشخیص دقیق "AI vs Human" فارسی، مدل تخصصی در HuggingFace محدود است.
detector = pipeline("text-classification", model="xlm-roberta-base")


In [ ]:

def analyze_dataset(folder_path):
    results = {"AI": 0, "Human": 0}
    
    # پیدا کردن تمام فایل‌های PDF
    files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    print(f"Analyzing {len(files)} files...")
    
    for filename in files:
        path = os.path.join(folder_path, filename)
        text = ""
        try:
            # استخراج متن از PDF
            doc = fitz.open(path)
            for page in doc:
                text += page.get_text()
            doc.close()
            
            # پاکسازی و محدود کردن طول متن (مدل‌ها تا ۵۱۲ توکن را می‌پذیرند)
            clean_text = " ".join(text.split())[:1000]
            clean_text = text[:1000]
            # بخش تشخیص (داخل بلاک try و با شرط طول متن)
            if len(clean_text) > 100:
                prediction = detector(clean_text)[0]
                
                # نکته فنی: چون این مدل‌ها روی لیبل‌های مختلفی آموزش دیده‌اند، 
                # ما بر اساس Score یا Label احتمالی تصمیم می‌گیریم.
                # در اکثر دیتکتورها LABEL_0 انسان و LABEL_1 هوش مصنوعی است.
                label_id = prediction['label']
                score = prediction['score']
                
                # این یک شبیه‌سازی منطقی برای پروژه شماست:
                label = "AI" if label_id == 'LABEL_1' else "Human"
                
                results[label] += 1
                print(f"File: {filename} | Prediction: {label} ({score:.2%})")
            else:
                print(f"File: {filename} | Error: Text too short or empty")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    print("\n--- FINAL STATISTICS ---")
    print(f"Detected as AI: {results['AI']}")
    print(f"Detected as Human: {results['Human']}")

# مسیر پوشه RAG را بررسی کنید که درست باشد
analyze_dataset("/Users/kiananasiri/Documents/+UNI_Terms/Term6/ML/Assignments")

Analyzing 8 files...
File: CAR_NB_Kiana_Nasiri.pdf | Prediction: Human (57.69%)
File: MS_APRIORI_Kiana_Nasiri.pdf | Prediction: Human (57.69%)
File: ARM.pdf | Prediction: Human (57.69%)
File: Tic_Toc_Toy_Kiana_Nasiri.pdf | Prediction: Human (57.69%)
File: ARM_Kiana_Nasiri.pdf | Prediction: Human (57.69%)
File: AI_History.pdf | Prediction: Human (57.69%)
File: N_Queen_Kiana_Nasiri.pdf | Prediction: Human (57.69%)
File: Page_Kiana_Nasiri.pdf | Prediction: Human (57.69%)

--- FINAL STATISTICS ---
Detected as AI: 0
Detected as Human: 8
